## Environment Preparation

In [2]:
%pip install python-dotenv > /dev/null 2>&1
%pip install datasets > /dev/null 2>&1
%pip install nltk > /dev/null 2>&1
%pip install openai > /dev/null 2>&1
%pip install tenacity > /dev/null 2>&1
%pip install scikit-learn > /dev/null 2>&1
%pip install pandas > /dev/null 2>&1
%pip install transformers > /dev/null 2>&1
%pip install torch > /dev/null 2>&1

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [26]:
from dotenv import load_dotenv

# Load the .env file
load_dotenv()

True

## Read Data From CSV

In [9]:
from datasets import load_dataset

same_lang_same_meaning_file = "./same_lang_same_meaning.csv"
same_lang_same_meaning_ds = load_dataset('csv', data_files=same_lang_same_meaning_file)['train']

same_lang_diff_meaning_file = "./same_lang_diff_meaning.csv"
same_lang_diff_meaning_ds = load_dataset('csv', data_files=same_lang_diff_meaning_file)['train']


diff_lang_same_meaning_file = "./diff_lang_same_meaning.csv"
diff_lang_same_meaning_ds = load_dataset('csv', data_files=diff_lang_same_meaning_file)['train']

diff_lang_diff_meaning_file = "./diff_lang_diff_meaning.csv"
diff_lang_diff_meaning_ds = load_dataset('csv', data_files=diff_lang_diff_meaning_file)['train']

all_datasets = [{
    "name": "same_lang_same_meaning",
    "ds": same_lang_same_meaning_ds
}, {
    "name": "same_lang_diff_meaning",
    "ds": same_lang_diff_meaning_ds
}, {
    "name": "diff_lang_same_meaning",
    "ds": diff_lang_same_meaning_ds
}, {
    "name": "diff_lang_diff_meaning",
    "ds": diff_lang_diff_meaning_ds
}]

## Testing 

Compare the effectiveness of different methods for evaluating sentence similarity, including word embeddings, lexical overlap, and semantic similarity.

In [22]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from typing import Callable, Union
import pandas as pd

# Calculate the cosine similarity between two embeddings
def calculate_cosine_similarity(embedding1: Union[np.ndarray,list[float]], embedding2: Union[np.ndarray, list[float]]) -> float:
    cosine_sim = cosine_similarity(np.array([embedding1]), np.array([embedding2]))
    return cosine_sim[0][0]

# Evaluate the embeddings using all the datasets
def evaluate_all_datasets(emb_func: Callable[[str], Union[np.ndarray, list[float]]]) -> Union[pd.DataFrame, None]:

    results = None

    for dataset in all_datasets:

        df = pd.DataFrame(columns=['sentence1', 'sentence2', 'cosine_similarity', 'expected_similarity', 'similarity_diff', 'source'])

        for i in range(0, len(dataset['ds'])):
            sentence1 = dataset['ds'][i]['sentence1']
            sentence2 = dataset['ds'][i]['sentence2']
            expected_similarity = dataset['ds'][i]['expected_similarity_score']
            
            embedding1 = emb_func(sentence1)
            embedding2 = emb_func(sentence2)

            similarity = calculate_cosine_similarity(embedding1, embedding2)

            similarity_diff = abs(similarity - expected_similarity)

            df.loc[i] = [sentence1, sentence2, similarity, expected_similarity, similarity_diff, dataset['name']]

        sorted_df = df.sort_values(by=['similarity_diff'], ascending=False)

        results = sorted_df if results is None else pd.concat([results, sorted_df])

    return results

#### BERT Embeddings

In [34]:
from transformers import BertTokenizer, BertModel
import torch

# Use the BERT multilingual model
model_name = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Get the embeddings of a sentence with BERT
def get_embedding_with_bert(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Get the sentence embedding
    sentence_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
    return sentence_embedding


bert_results = evaluate_all_datasets(get_embedding_with_bert)

# Display the results grouped by the source
if bert_results is not None:
    grouped_results = bert_results.groupby("source")
    for name, group in grouped_results:
        print(f"Source: {name}")
        display(group.style.hide("source", axis="columns"))
        print("\n")

Source: diff_lang_diff_meaning


,sentence1,sentence2,cosine_similarity,expected_similarity,similarity_diff
3,This is a red apple.,這是一顆綠蘋果。,0.925974,0.100000,0.825974
0,The weather is nice today.,今天的天氣很糟糕。,0.886153,0.100000,0.786153
5,Cats are cute animals.,狗是很可愛的動物。,0.870124,0.100000,0.770124
9,The conference starts tomorrow.,會議昨天開始。,0.840261,0.100000,0.740261
4,I enjoy reading books.,我討厭看書。,0.811470,0.200000,0.611470
1,I am going to the store.,我打算去學校。,0.757312,0.200000,0.557312
8,This is a complex problem.,這是一個簡單的問題。,0.734669,0.200000,0.534669
6,I like coffee in the morning.,我早上不喝咖啡。,0.692436,0.200000,0.492436
7,"Open the door, please.",請關上門。,0.482182,0.100000,0.382182
2,How are you doing?,你今天吃了什麼？,0.355033,0.300000,0.055033




Source: diff_lang_same_meaning


,sentence1,sentence2,cosine_similarity,expected_similarity,similarity_diff
2,How are you doing?,你最近怎麼樣？,0.342225,0.850000,0.507775
7,"Open the door, please.",請打開門。,0.605323,0.900000,0.294677
6,I like coffee in the morning.,我早上喜歡喝咖啡。,0.704720,0.950000,0.245280
1,I am going to the store.,我打算去商店。,0.771003,0.900000,0.128997
4,I enjoy reading books.,我喜歡看書。,0.822287,0.850000,0.027713
5,Cats are cute animals.,貓是很可愛的動物。,0.874766,0.900000,0.025234
3,This is a red apple.,這是一顆紅蘋果。,0.925271,0.950000,0.024729
9,She loves painting.,她喜歡畫畫。,0.878248,0.900000,0.021752
8,The car is blue.,這輛車是藍色的。,0.921708,0.900000,0.021708
0,The weather is nice today.,今天天氣很好。,0.893197,0.900000,0.006803




Source: same_lang_diff_meaning


,sentence1,sentence2,cosine_similarity,expected_similarity,similarity_diff
8,The car is blue.,The car is red.,0.989648,0.100000,0.889648
4,Cats are cute animals.,Cats are dangerous animals.,0.953387,0.100000,0.853387
0,The weather is nice today.,The weather is terrible today.,0.948222,0.100000,0.848222
5,I like coffee in the morning.,I dislike coffee in the morning.,0.926186,0.100000,0.826186
9,She loves painting.,She hates painting.,0.925527,0.100000,0.825527
3,I enjoy reading books.,I hate reading books.,0.909284,0.100000,0.809284
7,This is a complex problem.,This is an easy problem.,0.878860,0.100000,0.778860
6,"Open the door, please.","Close the door, please.",0.965688,0.200000,0.765688
2,How are you doing?,What is your name?,0.812617,0.100000,0.712617
1,I am going to the store.,I am staying at home.,0.890766,0.200000,0.690766




Source: same_lang_same_meaning


,sentence1,sentence2,cosine_similarity,expected_similarity,similarity_diff
1,I am going to the store.,I'm going to the shop.,0.937702,0.800000,0.137702
7,This is a complex problem.,This issue is complicated.,0.764714,0.900000,0.135286
5,I like coffee in the morning.,I enjoy drinking coffee in the morning.,0.827560,0.950000,0.122440
8,The car is blue.,The vehicle is blue.,0.987182,0.900000,0.087182
0,The weather is nice today.,The weather is great today.,0.984898,0.900000,0.084898
6,"Open the door, please.",Please open the door.,0.880698,0.950000,0.069302
4,Cats are cute animals.,Kittens are adorable creatures.,0.903702,0.850000,0.053702
9,She loves painting.,She enjoys creating art.,0.917933,0.900000,0.017933
3,I enjoy reading books.,I love to read novels.,0.864436,0.850000,0.014436
2,How are you doing?,How have you been?,0.850631,0.850000,0.000631


#### Azure OpenAI Embeddings

In [27]:
import os
from openai import AzureOpenAI
from tenacity import retry, stop_after_attempt, wait_exponential

aoai = AzureOpenAI(
            azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT", ""),
            api_key=os.environ.get("AZURE_OPENAI_KEY", ""),
            api_version="2024-02-01",
        )


@retry(stop=stop_after_attempt(5), wait=wait_exponential(multiplier=1, min=4, max=10))
def get_embedding_with_aoai(input: str, deployment_id: str) -> list[float]:
    return aoai.embeddings.create(
        input=input,
        model=deployment_id,
    ).data[0].embedding


##### Text Embedding Ada 002

In [29]:
from IPython.display import display

# Replace the deployment_id with the one you have created
text_embedding_ada = "text-embedding-ada-002"

text_embedding_ada_results = evaluate_all_datasets(lambda sentence: get_embedding_with_aoai(sentence, text_embedding_ada))

# Display the results grouped by the source
if text_embedding_ada_results is not None:
    grouped_results = text_embedding_ada_results.groupby("source")
    for name, group in grouped_results:
        print(f"Source: {name}")
        display(group.style.hide("source", axis="columns"))
        print("\n")

Source: diff_lang_diff_meaning


,sentence1,sentence2,cosine_similarity,expected_similarity,similarity_diff
3,This is a red apple.,這是一顆綠蘋果。,0.876102,0.100000,0.776102
7,"Open the door, please.",請關上門。,0.853319,0.100000,0.753319
9,The conference starts tomorrow.,會議昨天開始。,0.841885,0.100000,0.741885
0,The weather is nice today.,今天的天氣很糟糕。,0.826094,0.100000,0.726094
5,Cats are cute animals.,狗是很可愛的動物。,0.819996,0.100000,0.719996
4,I enjoy reading books.,我討厭看書。,0.823001,0.200000,0.623001
6,I like coffee in the morning.,我早上不喝咖啡。,0.819456,0.200000,0.619456
1,I am going to the store.,我打算去學校。,0.780782,0.200000,0.580782
8,This is a complex problem.,這是一個簡單的問題。,0.779760,0.200000,0.579760
2,How are you doing?,你今天吃了什麼？,0.779377,0.300000,0.479377




Source: diff_lang_same_meaning


,sentence1,sentence2,cosine_similarity,expected_similarity,similarity_diff
6,I like coffee in the morning.,我早上喜歡喝咖啡。,0.863240,0.950000,0.086760
3,This is a red apple.,這是一顆紅蘋果。,0.900953,0.950000,0.049047
2,How are you doing?,你最近怎麼樣？,0.810721,0.850000,0.039279
1,I am going to the store.,我打算去商店。,0.868281,0.900000,0.031719
9,She loves painting.,她喜歡畫畫。,0.870491,0.900000,0.029509
5,Cats are cute animals.,貓是很可愛的動物。,0.872485,0.900000,0.027515
4,I enjoy reading books.,我喜歡看書。,0.870658,0.850000,0.020658
8,The car is blue.,這輛車是藍色的。,0.882268,0.900000,0.017732
0,The weather is nice today.,今天天氣很好。,0.883527,0.900000,0.016473
7,"Open the door, please.",請打開門。,0.885239,0.900000,0.014761




Source: same_lang_diff_meaning


,sentence1,sentence2,cosine_similarity,expected_similarity,similarity_diff
5,I like coffee in the morning.,I dislike coffee in the morning.,0.930955,0.100000,0.830955
9,She loves painting.,She hates painting.,0.926090,0.100000,0.826090
8,The car is blue.,The car is red.,0.917279,0.100000,0.817279
4,Cats are cute animals.,Cats are dangerous animals.,0.908847,0.100000,0.808847
3,I enjoy reading books.,I hate reading books.,0.900507,0.100000,0.800507
0,The weather is nice today.,The weather is terrible today.,0.897225,0.100000,0.797225
6,"Open the door, please.","Close the door, please.",0.953652,0.200000,0.753652
2,How are you doing?,What is your name?,0.847203,0.100000,0.747203
7,This is a complex problem.,This is an easy problem.,0.845606,0.100000,0.745606
1,I am going to the store.,I am staying at home.,0.839788,0.200000,0.639788




Source: same_lang_same_meaning


,sentence1,sentence2,cosine_similarity,expected_similarity,similarity_diff
1,I am going to the store.,I'm going to the shop.,0.944095,0.800000,0.144095
4,Cats are cute animals.,Kittens are adorable creatures.,0.937997,0.850000,0.087997
3,I enjoy reading books.,I love to read novels.,0.931572,0.850000,0.081572
0,The weather is nice today.,The weather is great today.,0.977624,0.900000,0.077624
2,How are you doing?,How have you been?,0.926711,0.850000,0.076711
8,The car is blue.,The vehicle is blue.,0.970575,0.900000,0.070575
9,She loves painting.,She enjoys creating art.,0.947793,0.900000,0.047793
7,This is a complex problem.,This issue is complicated.,0.930616,0.900000,0.030616
5,I like coffee in the morning.,I enjoy drinking coffee in the morning.,0.975462,0.950000,0.025462
6,"Open the door, please.",Please open the door.,0.967786,0.950000,0.017786


##### Text Embedding 3 Small

In [30]:
from IPython.display import display


# Replace the deployment_id with the one you have created
text_embedding_3_small = "text-embedding-3-small"

text_embedding_3_small_results = evaluate_all_datasets(lambda sentence: get_embedding_with_aoai(sentence, text_embedding_3_small))

# Display the results grouped by the source
if text_embedding_3_small_results is not None:
    grouped_results = text_embedding_3_small_results.groupby("source")
    for name, group in grouped_results:
        print(f"Source: {name}")
        display(group.style.hide("source", axis="columns"))
        print("\n")

Source: diff_lang_diff_meaning


,sentence1,sentence2,cosine_similarity,expected_similarity,similarity_diff
7,"Open the door, please.",請關上門。,0.622425,0.100000,0.522425
3,This is a red apple.,這是一顆綠蘋果。,0.586463,0.100000,0.486463
5,Cats are cute animals.,狗是很可愛的動物。,0.503715,0.100000,0.403715
9,The conference starts tomorrow.,會議昨天開始。,0.498021,0.100000,0.398021
0,The weather is nice today.,今天的天氣很糟糕。,0.490473,0.100000,0.390473
6,I like coffee in the morning.,我早上不喝咖啡。,0.530172,0.200000,0.330172
4,I enjoy reading books.,我討厭看書。,0.453847,0.200000,0.253847
8,This is a complex problem.,這是一個簡單的問題。,0.378341,0.200000,0.178341
1,I am going to the store.,我打算去學校。,0.357277,0.200000,0.157277
2,How are you doing?,你今天吃了什麼？,0.290207,0.300000,0.009793




Source: diff_lang_same_meaning


,sentence1,sentence2,cosine_similarity,expected_similarity,similarity_diff
2,How are you doing?,你最近怎麼樣？,0.418580,0.850000,0.431420
6,I like coffee in the morning.,我早上喜歡喝咖啡。,0.662013,0.950000,0.287987
8,The car is blue.,這輛車是藍色的。,0.613637,0.900000,0.286363
5,Cats are cute animals.,貓是很可愛的動物。,0.613958,0.900000,0.286042
1,I am going to the store.,我打算去商店。,0.614360,0.900000,0.285640
9,She loves painting.,她喜歡畫畫。,0.659965,0.900000,0.240035
3,This is a red apple.,這是一顆紅蘋果。,0.726135,0.950000,0.223865
0,The weather is nice today.,今天天氣很好。,0.679327,0.900000,0.220673
4,I enjoy reading books.,我喜歡看書。,0.639054,0.850000,0.210946
7,"Open the door, please.",請打開門。,0.692977,0.900000,0.207023




Source: same_lang_diff_meaning


,sentence1,sentence2,cosine_similarity,expected_similarity,similarity_diff
5,I like coffee in the morning.,I dislike coffee in the morning.,0.825793,0.100000,0.725793
9,She loves painting.,She hates painting.,0.790465,0.100000,0.690465
6,"Open the door, please.","Close the door, please.",0.800257,0.200000,0.600257
8,The car is blue.,The car is red.,0.648949,0.100000,0.548949
4,Cats are cute animals.,Cats are dangerous animals.,0.631191,0.100000,0.531191
3,I enjoy reading books.,I hate reading books.,0.620641,0.100000,0.520641
0,The weather is nice today.,The weather is terrible today.,0.606137,0.100000,0.506137
7,This is a complex problem.,This is an easy problem.,0.593079,0.100000,0.493079
2,How are you doing?,What is your name?,0.427041,0.100000,0.327041
1,I am going to the store.,I am staying at home.,0.422274,0.200000,0.222274




Source: same_lang_same_meaning


,sentence1,sentence2,cosine_similarity,expected_similarity,similarity_diff
3,I enjoy reading books.,I love to read novels.,0.650389,0.850000,0.199611
9,She loves painting.,She enjoys creating art.,0.733806,0.900000,0.166194
2,How are you doing?,How have you been?,0.695640,0.850000,0.154360
4,Cats are cute animals.,Kittens are adorable creatures.,0.703765,0.850000,0.146235
7,This is a complex problem.,This issue is complicated.,0.791813,0.900000,0.108187
6,"Open the door, please.",Please open the door.,0.865304,0.950000,0.084696
5,I like coffee in the morning.,I enjoy drinking coffee in the morning.,0.872402,0.950000,0.077598
8,The car is blue.,The vehicle is blue.,0.856542,0.900000,0.043458
1,I am going to the store.,I'm going to the shop.,0.774428,0.800000,0.025572
0,The weather is nice today.,The weather is great today.,0.882139,0.900000,0.017861


##### Text Embedding 3 Large

In [31]:
from IPython.display import display

# Replace the deployment_id with the one you have created
text_embedding_3_large = "text-embedding-3-large"

text_embedding_3_large_results = evaluate_all_datasets(lambda sentence: get_embedding_with_aoai(sentence, text_embedding_3_large))

# Display the results grouped by the source
if text_embedding_3_large_results is not None:
    grouped_results = text_embedding_3_large_results.groupby("source")
    for name, group in grouped_results:
        print(f"Source: {name}")
        display(group.style.hide("source", axis="columns"))
        print("\n")

Source: diff_lang_diff_meaning


,sentence1,sentence2,cosine_similarity,expected_similarity,similarity_diff
9,The conference starts tomorrow.,會議昨天開始。,0.631538,0.100000,0.531538
3,This is a red apple.,這是一顆綠蘋果。,0.579195,0.100000,0.479195
7,"Open the door, please.",請關上門。,0.486157,0.100000,0.386157
5,Cats are cute animals.,狗是很可愛的動物。,0.469382,0.100000,0.369382
6,I like coffee in the morning.,我早上不喝咖啡。,0.547793,0.200000,0.347793
0,The weather is nice today.,今天的天氣很糟糕。,0.444041,0.100000,0.344041
4,I enjoy reading books.,我討厭看書。,0.464701,0.200000,0.264701
1,I am going to the store.,我打算去學校。,0.415134,0.200000,0.215134
8,This is a complex problem.,這是一個簡單的問題。,0.339543,0.200000,0.139543
2,How are you doing?,你今天吃了什麼？,0.203799,0.300000,0.096201




Source: diff_lang_same_meaning


,sentence1,sentence2,cosine_similarity,expected_similarity,similarity_diff
2,How are you doing?,你最近怎麼樣？,0.500746,0.850000,0.349254
8,The car is blue.,這輛車是藍色的。,0.592728,0.900000,0.307272
1,I am going to the store.,我打算去商店。,0.627266,0.900000,0.272734
5,Cats are cute animals.,貓是很可愛的動物。,0.668293,0.900000,0.231707
6,I like coffee in the morning.,我早上喜歡喝咖啡。,0.719445,0.950000,0.230555
0,The weather is nice today.,今天天氣很好。,0.680048,0.900000,0.219952
3,This is a red apple.,這是一顆紅蘋果。,0.746685,0.950000,0.203315
9,She loves painting.,她喜歡畫畫。,0.697199,0.900000,0.202801
7,"Open the door, please.",請打開門。,0.704367,0.900000,0.195633
4,I enjoy reading books.,我喜歡看書。,0.655715,0.850000,0.194285




Source: same_lang_diff_meaning


,sentence1,sentence2,cosine_similarity,expected_similarity,similarity_diff
5,I like coffee in the morning.,I dislike coffee in the morning.,0.781697,0.100000,0.681697
9,She loves painting.,She hates painting.,0.701742,0.100000,0.601742
6,"Open the door, please.","Close the door, please.",0.742564,0.200000,0.542564
8,The car is blue.,The car is red.,0.625397,0.100000,0.525397
3,I enjoy reading books.,I hate reading books.,0.618448,0.100000,0.518448
0,The weather is nice today.,The weather is terrible today.,0.597584,0.100000,0.497584
4,Cats are cute animals.,Cats are dangerous animals.,0.595407,0.100000,0.495407
7,This is a complex problem.,This is an easy problem.,0.509899,0.100000,0.409899
2,How are you doing?,What is your name?,0.402773,0.100000,0.302773
1,I am going to the store.,I am staying at home.,0.405678,0.200000,0.205678




Source: same_lang_same_meaning


,sentence1,sentence2,cosine_similarity,expected_similarity,similarity_diff
2,How are you doing?,How have you been?,0.677795,0.850000,0.172205
3,I enjoy reading books.,I love to read novels.,0.682395,0.850000,0.167605
4,Cats are cute animals.,Kittens are adorable creatures.,0.685430,0.850000,0.164570
7,This is a complex problem.,This issue is complicated.,0.736574,0.900000,0.163426
9,She loves painting.,She enjoys creating art.,0.737950,0.900000,0.162050
6,"Open the door, please.",Please open the door.,0.841711,0.950000,0.108289
5,I like coffee in the morning.,I enjoy drinking coffee in the morning.,0.862703,0.950000,0.087297
8,The car is blue.,The vehicle is blue.,0.863132,0.900000,0.036868
1,I am going to the store.,I'm going to the shop.,0.813740,0.800000,0.013740
0,The weather is nice today.,The weather is great today.,0.886953,0.900000,0.013047
